**Setup**

Importing modules.

In [ ]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode

Loading in data from csv.

In [ ]:
DF_ORIGINAL = pd.read_csv("../1_data_gather/manga.csv")

Filling all null tag and genre features with zero, as that is their implicit value.

In [ ]:
ZEROABLE_NUMERIC_COLUMN_NAMES = list(DF_ORIGINAL.drop(["id","chapters","volumes","start_year","start_month","start_day","end_year","end_month","end_day"],axis=1).select_dtypes(include=['number']).columns.values)
DEFAULT_ZEROES = [0] * len(ZEROABLE_NUMERIC_COLUMN_NAMES)
NULL_MAP = dict(zip(ZEROABLE_NUMERIC_COLUMN_NAMES, DEFAULT_ZEROES))
ZEROED_DF = DF_ORIGINAL.fillna(value=NULL_MAP)

Computing target feature from data.

In [ ]:
SHOUNEN_TAG_PCT = ZEROED_DF["Shounen"]
SHOUJO_TAG_PCT = ZEROED_DF["Shoujo"]
SEINEN_TAG_PCT = ZEROED_DF["Seinen"]
JOSEI_TAG_PCT = ZEROED_DF["Josei"]

demo_col = []

for x in range(ZEROED_DF.shape[0]):
    demo_dict = {"Shounen": SHOUNEN_TAG_PCT[x],
                 "Shoujo": SHOUJO_TAG_PCT[x],
                 "Seinen": SEINEN_TAG_PCT[x],
                 "Josei": JOSEI_TAG_PCT[x]}
    if sum(demo_dict.values()) > 0:
         demo_col.append(max(zip(demo_dict.values(), demo_dict.keys()))[1])
    else:
        demo_col.append(None)
    
DEMO_ADDED_DF = ZEROED_DF.assign(demo_label = pd.Series(demo_col)).drop(["Shounen","Shoujo","Seinen","Josei"], axis=1)

**Full-dataset summaries**

Checking summary statistics on numeric features, excluding the media ID.

In [ ]:
# Display data frames interactively
init_notebook_mode(all_interactive=True)

# Computing summary statistics for numeric columns, excluding the ID, adding in an explicit percentage of null values.
summary_df = DEMO_ADDED_DF.drop("id", axis=1).describe(include=np.number)
summary_df.loc["pct_null"] = DEMO_ADDED_DF.isna().mean().map("{:.1%}".format)
summary_df = summary_df.transpose()
summary_df = summary_df[['pct_null'] + [x for x in summary_df.columns if x != 'pct_null']]
summary_df